# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

d:\anaconda3\envs\py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("./datasets/ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [4]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## Step3 创建Dataset

In [5]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./datasets/ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [6]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [7]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [8]:
for i in range(10):
    print(trainset[i])

('太偏僻，不过非常安静，还有酒店为了省钱，居然没开空调，感觉不太好！免费注册网站导航宾馆索', 1)
('前台的服待客人都是不耐的子,住酒店的大概都是地人士,教一些地的是的,度真是不好;另外,居然有忙打的服.....;房及其他施,算是可以.', 0)
('这个酒店是我住过的国内四星级酒店里最差的一间，服务态度和质量都很差，房间很小，早餐连点肉都没有，9：00结束，8：00到的，基本就没有吃的了，而且服务员也不给加！也许是因为旅游景点的问题，但是我们住过附近其他的三星级酒店，各方面都比这家好，价格还能便宜100多，建议千万别住！酒店里有一家公安局在办公，所以大部分都是公安局的客户什么的，所以对普通百姓的服务很差，墙壁隔音设计很差，因此很早就显得很嘈杂，根本没法休息！', 0)
('环境很好，服务也不错，推荐下次还会住这里', 1)
('服务极差,有时候没空调!服务态度恶略!没空调问服务员要风扇,1小时过去了,都没来.投诉后才马上送来.', 0)
('房间配置很差，真可惜太仓没有如家和锦江之星，下次不会再入住这家酒店', 0)
('对的起价格。还有就是安静。就是大床竟然是二张小床拼的。让人郁闷。就是看他安静。所以我每次来还是住这了。', 1)
('酒店很干净，房间不错，早餐一般，但是周围环境不好，基本上没有什么可以吃饭的地方。', 0)
('环境位置很好,房间情况尚可,早餐一般般,价格偏高了一些.', 1)
('据说是当地最好的酒店了，挂三。我们住的豪标。优点：酒店环境尚可，房间大。缺点：离景区有25分钟左右路程，位置不佳，尤其想要在古镇看夜景的话不建议住这里。', 1)


## Step5 创建Dataloader

In [9]:
import torch

tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/models--hfl--rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [10]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [11]:
next(enumerate(trainloader))[0]

0

In [12]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101, 1765, 4415,  ...,    0,    0,    0],
        [ 101, 3300, 1380,  ..., 8013, 8013,  102],
        [ 101, 6821, 3613,  ...,    0,    0,    0],
        ...,
        [ 101, 5307,  679,  ..., 3300, 3198,  102],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101, 3301, 1351,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
        1, 0, 1

## Step6 创建模型及优化器

In [13]:
from torch.optim import AdamW

model = AutoModelForSequenceClassification.from_pretrained("D:/pretrained_model/models--hfl--rbt3")

if torch.cuda.is_available():
    model = model.cuda()

d:\anaconda3\envs\py39\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="c

In [14]:
optimizer = AdamW(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [23]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [24]:
train()

ep: 0, global_step: 0, loss: 0.0003495222772471607
ep: 0, global_step: 100, loss: 0.0005694393184967339
ep: 0, global_step: 200, loss: 0.00430734409019351
ep: 0, acc: 0.89304119348526
ep: 1, global_step: 300, loss: 0.006294787395745516
ep: 1, global_step: 400, loss: 0.00044037969200871885
ep: 1, acc: 0.894329845905304
ep: 2, global_step: 500, loss: 0.0020225581247359514
ep: 2, global_step: 600, loss: 0.0011818524217233062
ep: 2, acc: 0.8853092789649963


## Step9 模型预测

In [17]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [18]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [19]:
pipe(sen)

[{'label': '好评！', 'score': 0.9921243786811829}]

In [20]:
sents = ['大家都觉得很好，我觉得一般',"服务员都不管我们的","我觉得还可以，下次应该还会来"]
pipe(sents)

[{'label': '好评！', 'score': 0.8626711964607239},
 {'label': '差评！', 'score': 0.5146051049232483},
 {'label': '好评！', 'score': 0.9872178435325623}]